In [4]:
class RegresionLinealMultiple:  
    import statsmodels.formula.api as smf
    
    def __init__(self, url=None, y=None, **kwargs):
        self.url = url
        self.y = y
        self.kwargs = kwargs
        
    def variables_x(self):
        l = []
        for i in self.kwargs:
            l.append(i)
        return l
        
    def join_variables_x(self):
        var_x = self.variables_x()
        modelo_variables = '+'.join(var_x)
        return modelo_variables
    
    def modelo_multiple(self):
        
        import pandas as pd
        import statsmodels.formula.api as smf
        
        data = pd.read_csv(self.url)     
        variables_x = self.y + '~' + self.join_variables_x()
        lm = smf.ols(formula = variables_x, data = data).fit()
        return lm
    
    def view_model(self):
        #import statsmodels.formula.api as smf
        view = self.modelo_multiple()
        return view.summary()              
        
    def parameters_pvalues_rsquared(self):
        model = self.modelo_multiple()
        print('Parametros del modelo:\n{}\n-----------'.format(model.params))
        print('P_valores del modelo:\n{}\n-----------'.format(model.pvalues))
        print('R_cuadrada del modelo:\n{}\n-----------'.format(model.rsquared))
        print('R_cuadrada ajustada del modelo:\n{}\n-----------'.format(model.rsquared_adj))
        #print('Según el modelo: Sales = {} + {}*"TV" + {}*"Newspaper" '.format(model.params[0], model.params[1], model.params[2]))
       
    def prediccion_y(self):
        
        import pandas as pd
        
        """
         Llamamos a la función 'variables_x()' para obtener los nombre de
         las variables x y pasarlas por la función 'predict'.         
        """
         
        model = self.modelo_multiple()
        data = pd.read_csv(self.url)
        y_pred = model.predict(data[self.variables_x()])
        return y_pred
    
    def error(self):
        
        import pandas as pd
        import numpy as np
        
        count_var = len(self.variables_x())
        data = pd.read_csv(self.url) 
        SSD = sum((data[self.y]-self.prediccion_y())**2)
        RSE = np.sqrt(SSD/(len(data)-count_var-1))
        e = RSE / np.mean(data[self.y])
        # La salida de esta función, detalla el porcentaje de error que no puede explicar el modelo, entre menor sea es mejor para el modelo.
        return e
    
    # Multicolinealidad
        
        

In [5]:
path = 'https://raw.githubusercontent.com/joanby/python-ml-course/master/datasets/ads/Advertising.csv'
regre_mult = RegresionLinealMultiple(url = path, y = 'Sales', TV = 'TV', Radio='Radio', Newspaper='Newspaper')

In [6]:
regre_mult.join_variables_x()

'TV+Radio+Newspaper'

In [65]:
class Multicolinealidad:
    
    def __init__(self, url=None, var_1=None, **kwargs):
        self.url = url
        self.kwargs = kwargs
        self.var_1 = var_1
               
    def variables(self):
        return self.var_1
    
    def variables_y(self):
        l = []
        for i in self.kwargs:
            l.append(i)
            
        var = '+'.join(l)
        
        m = self.var_1 + '~' + var
        return m
    
    def funcion_multi(self):
        
        import statsmodels.formula.api as smf
        import pandas as pd
        
        l = []
        for i in self.kwargs:
            l.append(i)
            
        var = '+'.join(l)
        
        m = self.var_1 + '~' + var

        data = pd.read_csv(self.url)
        lm = smf.ols(formula = m, data = data).fit()
        rsquare = lm.rsquared
        VIF = 1/(1-rsquare)
        # El VIF es el factor de inflación de la varianza, y se usa cuando se 
        # tienen problemas de multicolinealidad, esto se da cuando existe coorrelación entre
        # variables predictoras
        return VIF

In [86]:
url = 'https://raw.githubusercontent.com/joanby/python-ml-course/master/datasets/ads/Advertising.csv'
m = Multicolinealidad(url = url , var_1 = 'Newspaper', TV='TV', Radio='Radio')

In [87]:
m.variables()

'Newspaper'

In [88]:
m.variables_y()

'Newspaper~TV+Radio'

In [89]:
m.funcion_multi()

1.1451873787239286

In [73]:
class Validacion:
    
    def __init__(self, url=None, var_1=None, **kwargs):
        self.url = 'https://raw.githubusercontent.com/joanby/python-ml-course/master/datasets/ads/Advertising.csv'
        self.var_1 = var_1
        self.kwargs = kwargs  
        
    def variables_y(self):
        l = []
        for i in self.kwargs:
            l.append(i)
        var = '+'.join(l)
        m = self.var_1 + '~' + var
        return m
    
    def datos(self):
        import pandas as pd
        import numpy as np
        import statsmodels.formula.api as smf
        data = pd.read_csv(self.url)
        a = np.random.randn(len(data))
        check = (a<0.8)
        # Nos indica que el 80% de los datos se usaran para entrenar al modelo
        training = data[check]
        # Lo contrario de check, el 20% se usara para testing
        testing = data[~check]
        # Esta función compara los datos training vs testing
        lm = smf.ols(formula = self.variables_y(), data = training).fit()
        return lm
    
    def detalle(self):
        return self.datos().summary()
    
    
    def histograma(self):
        import matplotlib.pyplot as plt
        plt.hist(self.datos())
        # Dividir el dataset en conjunto de entrenamiento y de testing
        
    def predict_testing(self):
        import pandas as pd
        import numpy as np
        
        data = pd.read_csv(self.url)
        a = np.random.randn(len(data))
        check = (a<0.8)
        # Nos indica que el 80% de los datos se usaran para entrenar al modelo
        training = data[check]
        # Lo contrario de check, el 20% se usara para testing
        testing = data[~check]
        predict = self.datos().predict(testing)
        return predict
    
    def error(self):
        
        import pandas as pd
        import numpy as np
        
        data = pd.read_csv(self.url)
        a = np.random.randn(len(data))
        check = (a<0.8)
        # Nos indica que el 80% de los datos se usaran para entrenar al modelo
        training = data[check]
        # Lo contrario de check, el 20% se usara para testing
        testing = data[~check]
        predict = self.datos().predict(testing)
        SSD = sum((testing['Sales'] - predict)**2)
        
        l = []
        for i in self.kwargs:
            l.append(i)
        
        RSE = np.sqrt(SSD/(len(testing)-len(l)-1))
        
        var_y_mean = np.mean(testing[self.var_1])
        
        err = RSE/var_y_mean
        
        return err
        
        
    
        
    

In [74]:
v = Validacion(var_1 = 'Sales', TV='TV', Radio='Radio')
v.datos()

In [75]:
v.error()

0.1304402126052511

In [77]:
v.detalle()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     639.9
Date:                Tue, 11 Dec 2018   Prob (F-statistic):           4.60e-77
Time:                        20:20:17   Log-Likelihood:                -315.43
No. Observations:                 163   AIC:                             636.9
Df Residuals:                     160   BIC:                             646.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9817      0.331      9.009      0.000       2.328       3.635
TV             0.0457      0.002     29.053      0.000       0.043       0.049
Radio          0.1844      0.009     20.308      0.000       0.166       0.202
==============================================================================
Omnibus:                       57.277   Durbin-Watson:                   1.871
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              154.354
Skew:                          -1.449   Prob(JB):                     3.04e-34
Kurtosis:                       6.785   Cond. No.                         421.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Notas Importantes

* Un incremento en el estadistico F, es sinonimo de que las variables que se agregaron son mejor que las anteriores. Es sinonimo de una mejora en el modelo y quiere decir que esa nueva variable, predice mejor en conjunto que el modelo tal cual estaba sin ella.
* Cuanto menor sea el pvalor para estimar de las variables predictoras mejor es añadir esa variable predictora al modelo 
* El RSE es el valor estandar de los residuos, cada que se añade una variable nueva, este indicador decrece, lo que indica una m
* El error es lo que el modelo no puede explicar y se representa en porcentaje, este nos indica que existe x% de datos que no pueden ser explicados, entre menor sea es mejor para el modelo.

